# <center> <img src="../../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> Computer Systems Engineering  </center>
---
### <center> Big Data Processing </center>
---
#### <center> **Autumn 2025** </center>

#### <center> **Final Project: Machine Learning** </center>
---

**Date**: October, 2025

**Student Name**: Axel Gallardo

**Professor**: Pablo Camarillo Ramirez

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, round as spark_round

# Crear sesión de Spark
spark = SparkSession.builder \
    .appName("ML project") \
    .master("spark://spark-master:7077") \
    .config("spark.sql.shuffle.partitions", "5") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

sc = spark.sparkContext
sc.setLogLevel("INFO")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/22 19:24:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Machine Learning algorithm to use

En este proyecto el objetivo será predecir el nivel de desempeño académico de un estudiante a partir de sus características personales y hábitos de estudio usando un modelo de clasificación con Random Forest debido a que maneja muy bien relaciones no lineales y variables numéricas codificadas como enteros.


# Dataset Description
Para este proyecto utilizo el conjunto de datos “Students Performance Dataset” disponible públicamente en Kaggle aquí https://www.kaggle.com/datasets/rabieelkharoua/students-performance-dataset

El dataset contiene información de 2,392 estudiantes en un CSV de aproximadamente 167 KB organizados en 15 columnas:

StudentID

Age

Gender

Ethnicity

ParentalEducation

ParentalSupport

StudyTimeWeekly

Absences

Tutoring

Extracurricular

Sports

Music

Volunteering

GPA

GradeClass

A continuación se demuestra que el dataset no está balanceado basado en la variable objetivo (GradeClass)

In [2]:
from pcamarillor.spark_utils import SparkUtils

# Definir el esquema del dataset
student_schema = SparkUtils.generate_schema([
    ("StudentID",         "int"),
    ("Age",               "int"),
    ("Gender",            "int"),
    ("Ethnicity",         "int"),
    ("ParentalEducation", "int"),
    ("StudyTimeWeekly",   "float"),
    ("Absences",          "int"),
    ("Tutoring",          "int"),
    ("ParentalSupport",   "int"),
    ("Extracurricular",   "int"),
    ("Sports",            "int"),
    ("Music",             "int"),
    ("Volunteering",      "int"),
    ("GPA",               "float"),
    ("GradeClass",        "float")
])

student_df = spark.read \
    .option("header", "true") \
    .schema(student_schema) \
    .csv("/opt/spark/work-dir/data/ProyectoML/")

total_rows = student_df.count()
print(f"Total de registros: {total_rows}")

balance_df = (
    student_df
      .groupBy("GradeClass")
      .agg(count("*").alias("count"))
      .withColumn(
          "percentage",
          spark_round(col("count") / total_rows * 100, 2)
      )
      .orderBy("GradeClass")
)

print("Distribución de la variable objetivo (GradeClass):")
balance_df.show(truncate=False)


Total de registros: 2392
Distribución de la variable objetivo (GradeClass):


+----------+-----+----------+
|GradeClass|count|percentage|
+----------+-----+----------+
|0.0       |107  |4.47      |
|1.0       |269  |11.25     |
|2.0       |391  |16.35     |
|3.0       |414  |17.31     |
|4.0       |1211 |50.63     |
+----------+-----+----------+



# ML Training process

Para entrenar el modelo se reutilizó el flujo de el notebook "Lecture 19 decision trees random forest", ajustando los features para ser todas las columnas menos el id y las 2 columnas que serían las objetivo: GPA y GradeClass.

La división de entrenamiento sigue siendo 80-20 y la semilla usada para mantener los resultados constantes a cada ejecución fue la 10, inicialmente el numero de arboles y la profundidad eran 3 y 5, pero al necesitar de resultados más precisos se aumentó a 50 y 10, y finalmente se redujo a 25 y 8 por alertas de tareas con un tamaño excesivo

Finalmente se guardó el modelo en la ruta "/opt/spark/work-dir/data/mlmodels/student_performance/rf_gradeclass_model"

In [21]:
from pyspark.ml.feature import VectorAssembler

feature_cols = [
    "Age",
    "Gender",
    "Ethnicity",
    "ParentalEducation",
    "StudyTimeWeekly",
    "Absences",
    "Tutoring",
    "ParentalSupport",
    "Extracurricular",
    "Sports",
    "Music",
    "Volunteering"
]

assembler = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features"
)

data_with_features = assembler.transform(student_df) \
                               .select("GradeClass", "features") \
                               .withColumnRenamed("GradeClass", "label")

train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=10)

from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(
    labelCol="label",
    featuresCol="features",
    numTrees=25,
    maxDepth=8,
    seed=10
)

rf_model = rf.fit(train_df)

model_path = "/opt/spark/work-dir/data/mlmodels/student_performance/rf_gradeclass_model"

rf_model.write().overwrite().save(model_path)

# ML Evaluation
Para la evaluación se usó el conjunto de prueba previamente generado, y como resultado se obtuvo una precisión y un puntaje F1 de cerca del 70%, resultado similar al obtenido con 50 arboles de 10 de profundidad, por lo que su reducción se vió adecuada.

Se debe tener más en cuenta el puntaje de F1 pues al estar desbalanceado el modelo este representa mejor el puntaje de precisión obtenido.

Todo esto aplicado al modelo guardado para comprobar que fue correctamente almacenado.

In [22]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

rf_model_saved = rf_model.load(model_path)

predictions = rf_model_saved.transform(test_df)

evaluator = MulticlassClassificationEvaluator(
    labelCol="label",
    predictionCol="prediction"
)

accuracy = evaluator.evaluate(predictions, 
                  {evaluator.metricName: "accuracy"})
print(f"Accuracy of Random Forest: {accuracy}")

f1 = evaluator.evaluate(predictions,
                {evaluator.metricName: "f1"})
print(f"F1 Score of Random Forest: {f1}") 


Accuracy of Random Forest: 0.6942148760330579
F1 Score of Random Forest: 0.6809299888442133


In [23]:
sc.stop()